# Evaluation der Trainingskurven und der Ergebnisse auf dem Testset

Genau wie im Trainings-Notebook importieren wir zunächst einige Bibliotheken, die wir benutzen werden, richten unsere Umgebung ein und wählen die Graphikkarte, die wir für unsere Berechnungen benutzen.

In [ ]:
# PyTorch ist neben TensorFlow eines der zwei großen "Backends" für
# Deep-Learning, d.h. eine der Bibliotheken die die zu Grunde 
# liegende Funktionalität bereit stellen, um tiefe neuronale Netze
# zu Trainieren und Anzuwenden. Man importiert PyTorch 
# und verschiedene Sub-Module wie folgt:
import torch

# Und wir benutzen wieder matplotlib zur graphischen Darstellung, in 
# diesem Fall der Lernkurven
import matplotlib.pyplot as plt 

# Wir testen im folgenden, ob und wieviele Graphikkarten auf dem Notebook-Server zur Verfügung
# stehen und wählen eine davon aus (bitte nicht ändern, sonst kommen sie eventuell
# ihren Mitstudierenden ins Gehege).

gpu_nr = 0
if torch.cuda.is_available():
    print('%d GPU(s) available.' % torch.cuda.device_count())
    
    if torch.cuda.device_count() > gpu_nr:
        device = "cuda:%d" % gpu_nr 
    else:
        device = "cpu"
else:
    device = "cpu"

print('Using device: ' + device)

### Trainingskurven
Zunächst schauen wir uns die Trainingskurven an. Falls z.B. die Learning-Rate zu groß ist und die Kostenfunktion explodiert, kann man das so sofort erkennen.

__Ohne eine Darstellung der Lernkurven, die zeigen, wie sich die Kostenfunktion auf dem Trainings- und Validierungsset während des Trainings entwickelt, sollte man keinem Deep-Learning-Ergebnis trauen.__

In [ ]:
# Wir teilen matplotlib mit, dass die Kurven direkt hier ins Notebook gezeichnet werden sollen.
%matplotlib inline

# Jetzt lesen wir die Metriken, die wir in der Trainingsschleife nach jeder Epoche gespeichert haben
from helper_functions_covid import read_log_data

log_file = 'log_train.txt'
log_data = read_log_data(log_file)

# Zuerst zeichnen wir eine Graphik, die die Lernkurven der Kostenfunktion ("loss") auf dem Trainings-
# und Validierungsset darstellt
plt.figure()
plt.title('Loss')
plt.plot(log_data['epoch'],log_data['train loss'],label="train loss")
plt.plot(log_data['epoch'],log_data['validation loss'],label="validation loss")
plt.xlabel('Epoche')
plt.ylabel('Kostenfunktion')
plt.legend(loc='best')
plt.show()

# Jetzt zeichnen wir noch eine Graphik, die die Lernkurven der Accuracy auf dem Trainings-
# und Validierungsset darstellt
plt.figure()
plt.title('Accuracy')
plt.plot(log_data['epoch'],log_data['train accuracy'],label="train accuracy")
plt.plot(log_data['epoch'],log_data['validation accuracy'],label="validation accuracy")
plt.legend(loc='best')
plt.xlabel('Epoche')
plt.ylabel('Accuracy')
plt.show()

### Metriken auf dem Testset

Als nächstes schauen wir uns die Kennwerte des trainierten Netzwerkes auf dem unabhängigen Testset an, das wir bereits im Trainings-Notebook vorbereitet und gespeichert haben.

Dazu laden wir zunächst die Dateinamen aus den entsprechenden Listen, die wir im Trainings-Notebook erzeugt haben.

In [ ]:
from helper_functions_covid import read_txt

filenames_normal_test = read_txt('filenames_normal_test.txt')
filenames_pneumonia_test = read_txt('filenames_pneumonia_test.txt')
filenames_covid_test = read_txt('filenames_covid_test.txt')

Jetzt erzeugen wir einen entsprechenden Datensatz, der diese Dateien so lädt und aufbereitet, dass das trainierte Netz sie verarbeiten kann.

In [ ]:
from helper_functions_covid import ImageDataset, val_transformer, DataLoader

data_path = './covid_dataset/'
batchsize = 8

testset = ImageDataset(root_dir=data_path,
                          classes = ['normal', 'pneumonia', 'COVID'],
                          files_path = [filenames_normal_test, filenames_pneumonia_test, filenames_covid_test],
                          transform= val_transformer)

test_loader = DataLoader(testset, batch_size=batchsize, drop_last=False, shuffle=False)

Nun können wir die Metriken auf dem Testset evaluieren.

In [ ]:
from helper_functions_covid import compute_metrics

model = torch.load("best_model.pkl")

metrics_dict = compute_metrics(model, test_loader, device)
# Wir geben die entsprechenden Statistiken aus:
print('------------------ Test Metrics ----------------------------------------------')
print("Accuracy \t {:.3f}".format(metrics_dict['accuracy']))
print("Recall on normal test data \t {:.3f}".format(metrics_dict['recall_normal']))
print("Recall on pneumonia test data \t {:.3f}".format(metrics_dict['recall_pneumonia']))
print("Recall on COVID test data \t {:.3f}".format(metrics_dict['recall_covid']))
print("Test Loss \t {}".format(metrics_dict["validation loss"]))
print("------------------------------------------------------------------------------")

Nun erstellen wir auch noch die __"Confusion Matrix"__ für das trainierte Netzwerk auf dem Testset.

In [ ]:
from helper_functions_covid import plot_confusion_matrix

conf_matrix = metrics_dict["confusion matrix"]

plot_confusion_matrix(conf_matrix)

### GradCAM

Ein häufig (zu recht) geäußerter Kritikpunkt an Deep-Learning-Methoden ist, dass es sich bei den trainierten Netzen um "Black Boxes" handelt. Aufgrund der schieren Komplexität und der Anzahl der freien Parameter ist es - im Gegensatz zu regelbasierten Systemen oder klassischen Algorithmen - nicht einfach, herauszufinden, nach welchen Regeln oder Kriterien ein neuronales Netzwerk seine Entscheidungen fällt. 

Aufgrund der Bedeutung dieser Frage gibt es zum Glück einiges an Forschung und Fortschritt auf diesem Gebiet. 

Ein Ansatz ist __Feature Attribution__: Feature Attribution bedeutet zu untersuchen, welche "Features" eines Datenpunktes den Output eines Classifiers am stärksten beeinflussen. 

Bei der Klassifikation von Bilder kann man zum Beispiel untersuchen, welche Pixel eines Inputbildes die vorhergesagten Klassewahrscheinlichkeiten am stärksten beeinflussen würden, wenn man an diesen "ein wenig wackeln" würde.

Ein solcher Algorithmus ist z.B. [GradCAM](https://arxiv.org/abs/1610.02391), den wir auf unserem trainierten Netz ausprobieren wollen.

Besonders lehrreich ist das, wenn wir wissen, was die wirkliche Klasse eines Bildes ist, und welche Klasse das Netzwerk dem Datenpunkt zugeordnet hat.

In [ ]:
# Eventuell einmal pytorch-gradcam installieren und danach den Kernel nochmals neu starten, 
# falls diese Zelle nicht ausführbar sein sollte:
# !pip install pytorch-gradcam


from helper_functions_covid import show_grad_cam

# Vorhersagen, die das Netz für die Bilder des Testsets gemacht hat
preds = metrics_dict["pred_list"]
# Wahre Klassen für die Bilder des Testsets
targets = metrics_dict["target_list"]
# Pfade zu den Bildern des Testsets
paths = metrics_dict["paths"]

# Diese Funktion erhält neben den vorhergesagten Klassen, den wahren Klassen und den Dateinamen
# der Datenpunkte aus dem Testset (preds, targets, paths) auch das trainierte Netzwerk (model).
# Die Funktion berechnet auf der Graphikkarte (device) die entsprechenden Feature-Attribution-Maps für
# zufällig gezogene Beispiele aus dem Testset, die vom Netzwerk als "predicted_label" klassifiziert 
# wurden, in Wahrheit aber zur Klasse "true_label" gehören.
# D.h. wenn sie die Funktion so ausführen, zeigt sie Beispiele an, die als covid Klassifiziert wurden
# und auch wirklich zur Klasse "covid" gehören.
# Probieren sie gerne auch andere Kombinationen aus und beobachten sie, wie sich die Maps
# z.B. für korrekte und falsche Vorhersagen unterscheiden.
# Mögliche optionen für predicted_laben und true_label sind die Klassen unseres Datensatzes, also
# 'covid', 'normal' und 'pneumonia'
show_grad_cam(preds, targets, paths, model, device, predicted_label = 'covid', true_label = 'covid')

Das wars. Wir hoffen, sie konnten in dieser Übung etwas lernen. Laden sie gerne die entsprechenden Bilder (mit Rechtsklick) herunter und kopieren sie die entsprechenden Statistiken, um sie dann in das Miro-Board zu kopieren, das wir für die Abschlussbesprechung verwenden möchten (Link auf HeiCONF).